# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [10]:
from azureml.core import Workspace, Experiment, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import pandas as pd
import os
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# The data will be retrived by prain.py script, see below

In [2]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="red-wine-quality")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: nk_project3
Azure region: westeurope
Subscription id: 3dafc970-6015-4fba-b00d-39dc6a1b7521
Resource group: nk_capstoneproject


In [3]:
# test python script
!python train.py --C=0.6 --max_iter=11

Attempted to log scalar metric Regularization Strength::
0.6
Attempted to log scalar metric Max iterations::
11
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
Attempted to log scalar metric Accuracy:
0.8371212121212122


In [4]:
#Create compute

cpu_cluster_name = "natcluster001"
compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [11]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 2)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
    "--C": uniform(0.1, 1),
    "--max_iter": choice([50, 75, 100, 125])
    }
)

#TODO: Create your estimator and hyperdrive config

env_hd = Environment.get(workspace=ws, name="AzureML-Tutorial")

run_config_hd =ScriptRunConfig (source_directory = './',
compute_target = 'natcluster001',
script = 'train.py',
environment=env_hd)

hyperdrive_config = HyperDriveConfig(
run_config=run_config_hd,
hyperparameter_sampling = ps,
policy = policy,
primary_metric_name = 'Accuracy',
primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
max_total_runs = 12,
max_concurrent_runs = 4
)


In [12]:
# Submit experiment
hdr_run = exp.submit(config=hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
RunDetails(hdr_run).show()

hdr_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_053a5d21-0363-4a24-9c85-6e71e8a9688d
Web View: https://ml.azure.com/runs/HD_053a5d21-0363-4a24-9c85-6e71e8a9688d?wsid=/subscriptions/3dafc970-6015-4fba-b00d-39dc6a1b7521/resourcegroups/nk_capstoneproject/workspaces/nk_project3&tid=dd951004-cd14-4cb2-8718-3c0bcc7fed4e

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-08T15:16:37.609325][API][INFO]Experiment created<END>\n""<START>[2021-08-08T15:16:38.092535][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-08T15:16:38.376042][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_053a5d21-0363-4a24-9c85-6e71e8a9688d
Web View: https://ml.azure.com/runs/HD_053a5d21-0363-4a24-9c85-6e71e8a9688d?wsid=/subscriptions/3dafc970-6015-4fba-b00d-39dc6a1b7521/resourcegroups/nk_capstoneproject/workspaces/nk_project3&tid=dd951004-cd14-4cb2-8718-3c0bcc7fed4e



{'runId': 'HD_053a5d21-0363-4a24-9c85-6e71e8a9688d',
 'target': 'natcluster001',
 'status': 'Completed',
 'startTimeUtc': '2021-08-08T15:16:37.403294Z',
 'endTimeUtc': '2021-08-08T15:27:12.068138Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '5d7841a4-b7f9-4827-a275-1c9839d6ebe3',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9015151515151515',
  'best_child_run_id': 'HD_053a5d21-0363-4a24-9c85-6e71e8a9688d_5',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://nkproject31286776690.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_053a5d21-0363-4a24-9c85-6e71e8a9688d/azureml-logs/hyperdrive.txt?sv=2019-07

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [14]:
# Get the best run

best_hdr_run = hdr_run.get_best_run_by_primary_metric()

In [15]:
# Save the best model
hdr_model = best_hdr_run.register_model(model_name='wine-best-hdr-model2', model_path='./outputs/model_wine002.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service